# Tensorflow 一些基础知识

和pytorch之类的动态计算图不同，tensorflow就连把值打出来都要一番努力。搞得我Edward学习也是屡屡受挫

In [1]:
import tensorflow as tf

In [2]:
a = tf.constant(1)
b = tf.constant(2)
c = a + b


In [3]:
print(c)

Tensor("add:0", shape=(), dtype=int32)


可以看到c的值并没有被打出来，因为这里只不过在构造（描述）了个计算图而已，并没有进行任何计算。


In [4]:
c.eval()

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

可以看出你甚至不能直接计算它，尽管这似乎非常明显（theano好像可以）。你一定要用一个所谓的session去计算它。

In [5]:
sess = tf.Session()  

In [6]:
sess.run(c)

3

In [7]:
c.eval(session=sess)

3

In [9]:
sess.run([a,b,c])

[1, 2, 3]

可以看出，在tensorflow的魔性管理下，只有明确写出的计算图中的符号的计算值才会返回。那我们要历史值怎么办呢？手动for呗，
每轮run一次fetch个返回值。

人家PyMC3一个sample接口（蒙特卡洛模拟），一个fit接口（变分推断），返回一个足够强大的对象，不知道比你这个蹩脚集成高到哪里去了。

然而Edward里很多例子都是那个推断对象直接run迭代了好多次，然后结果也看不到，怎么破？

不用run呗。。用update。

当然计算图最后的值还是在的，但你要想用模拟法就自求多福吧。

PyMC3的变分推断是不用传q的，而你Edward就是蒙特卡洛模拟也需要绑定个不知所云的随机变量（一个Empirical，仿佛在嘲笑我的智商），
令人费解。

tensorflow那些符号叫op，就是run的那些，可以返回值得东西。有意思的是优化器也是op，另外看Edward源代码实现也爱自己加一些op上去。
看上去如果不使用主训练op，看些其他状态（变量）用sess.run开销也不大。

In [11]:
d = tf.Variable(0)

In [14]:
e = tf.assign_add(d,1)

In [15]:
sess.run([d,e])

FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[Node: AssignAdd = AssignAdd[T=DT_INT32, _class=["loc:@Variable"], use_locking=false, _device="/job:localhost/replica:0/task:0/cpu:0"](Variable, AssignAdd/value)]]

Caused by op 'AssignAdd', defined at:
  File "C:\Users\yiyuezhuo\Anaconda3\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\yiyuezhuo\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\yiyuezhuo\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\yiyuezhuo\Anaconda3\lib\site-packages\traitlets\config\application.py", line 596, in launch_instance
    app.start()
  File "C:\Users\yiyuezhuo\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\yiyuezhuo\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 162, in start
    super(

居然还不能运行，简直有毒。

In [18]:
sess.run(d.initializer)

In [19]:
sess.run([d,e])

[1, 1]

In [20]:
sess.run([d,e])

[2, 2]

In [21]:
sess.run([d,e])

[3, 3]